# MNE : From raw data to epochs and evoked responses (ERF/ERP)

`
Authors:
Alexandre Gramfort
Denis A. Engemann
`

`
Modified by:
Egor Ananyev
Jit Wei Ang Aaron
`

In [1]:
%matplotlib qt
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

First, load the _mne_ package:

In [2]:
import mne

We set the log-level to 'warning' so the output is less verbose

In [3]:
mne.set_log_level('warning')

### Remember if you need help just ask... the machine

In [4]:
mne.pick_types?

Signature:
mne.pick_types(
    info,
    meg=True,
    eeg=False,
    stim=False,
    eog=False,
    ecg=False,
    emg=False,
    ref_meg='auto',
    misc=False,
    resp=False,
    chpi=False,
    exci=False,
    ias=False,
    syst=False,
    seeg=False,
    dipole=False,
    gof=False,
    bio=False,
    ecog=False,
    fnirs=False,
    include=(),
    exclude='bads',
    selection=None,
)
Docstring:
Pick channels by type and names.

Parameters
----------
info : dict
    The measurement info.
meg : bool | str
    If True include all MEG channels. If False include None
    If string it can be 'mag', 'grad', 'planar1' or 'planar2' to select
    only magnetometers, all gradiometers, or a specific type of
    gradiometer.
eeg : bool
    If True include EEG channels.
stim : bool
    If True include stimulus channels.
eog : bool
    If True include EOG channels.
ecg : bool
    If True include ECG channels.
emg : bool
    If True include EMG channels.
ref_meg : bool | str
    If True incl

## Access raw data

You should have downloaded the `ds000117-practical` folder.

In [5]:
import os

# Change the following path to where the folder ds000117-practical is on your disk
# data_path = os.path.expanduser("~/work/data/ds000117-practical/")  # original path
# data_path = os.path.expanduser("~/Downloads/meeg/ds000117-practical/")  # this results in error for <ls $raw_fname>
# data_path = os.path.expanduser("C:/Users/egora/Downloads/meeg/ds000117-practical/")  # this results in error for <ls $raw_fname>
# data_path = os.path.expanduser("C:\\Users\\egora\\Downloads\\meeg\\ds000117-practical\\")  # this works
data_path = os.path.expanduser("~\\Downloads\\meeg\\ds000117-practical\\")  # this works and is user-independent

raw_fname = os.path.join(data_path,
    'derivatives\\meg_derivatives\\sub-01\\ses-meg\\meg\\sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

In [6]:
print(raw_fname)

C:\Users\egora\Downloads\meeg\ds000117-practical\derivatives\meg_derivatives\sub-01\ses-meg\meg\sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif


In [7]:
ls $raw_fname

 Volume in drive C is Acer
 Volume Serial Number is FAD5-0445

 Directory of C:\Users\egora\Downloads\meeg\ds000117-practical\derivatives\meg_derivatives\sub-01\ses-meg\meg

2019-12-12  08:27       879,788,831 sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif
               1 File(s)    879,788,831 bytes
               0 Dir(s)  88,336,199,680 bytes free


Read data from file:

In [8]:
mne.io.read_raw_fif?

Signature:
mne.io.read_raw_fif(
    fname,
    allow_maxshield=False,
    preload=False,
    verbose=None,
)
Docstring:
Reader function for Raw FIF data.

Parameters
----------
fname : str | file-like
    The raw filename to load. For files that have automatically been split,
    the split part will be automatically loaded. Filenames should end
    with raw.fif, raw.fif.gz, raw_sss.fif, raw_sss.fif.gz, raw_tsss.fif,
    raw_tsss.fif.gz, or _meg.fif. If a file-like object is provided,
    preloading must be used.

    .. versionchanged:: 0.18
       Support for file-like objects.
allow_maxshield : bool | str (default False)
    If True, allow loading of data that has been recorded with internal
    active compensation (MaxShield). Data recorded with MaxShield should
    generally not be loaded directly, but should first be processed using
    SSS/tSSS to remove the compensation signals that may also affect brain
    activity. Can also be "yes" to load without eliciting a warning.

prelo

In [9]:
raw = mne.io.read_raw_fif(raw_fname, preload=False)
print(raw)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 404 x 540100 (491.0 sec), ~7.4 MB, data not loaded>


Note the `preload=False` which states that no data is actually in memory.

For general info on importing MEG see:
https://mne.tools/stable/auto_tutorials/io/plot_10_reading_meg_data.html
or EEG see:
https://mne.tools/stable/auto_tutorials/io/plot_20_reading_eeg_data.html

Now let's look at the measurement info. It will give details about:

   - sampling rate
   - filtering parameters
   - available channel types
   - bad channels
   - etc.

In [10]:
mne.__version__

'0.19.2'

In [11]:
print(raw.info.keys())

dict_keys(['file_id', 'events', 'hpi_results', 'hpi_meas', 'subject_info', 'device_info', 'helium_info', 'hpi_subsystem', 'proc_history', 'meas_id', 'experimenter', 'description', 'proj_id', 'proj_name', 'meas_date', 'utc_offset', 'sfreq', 'highpass', 'lowpass', 'line_freq', 'gantry_angle', 'chs', 'dev_head_t', 'ctf_head_t', 'dev_ctf_t', 'dig', 'bads', 'ch_names', 'nchan', 'projs', 'comps', 'acq_pars', 'acq_stim', 'custom_ref_applied', 'xplotter_layout', 'kit_system_id'])


In [12]:
print(raw.info['meas_date'])  # EA: some sort of issue with dates, possibly with this field

(-908196946, 988669)


In [13]:
print(raw.info)

-908196946
<Info | 27 non-empty fields
    acq_pars : str | 21833 items
    bads : list | 0 items
    ch_names : list | MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, ...
    chs : list | 404 items (GRAD: 204, MAG: 102, EEG: 74, STIM: 3, MISC: 12, CHPI: 9)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 36 items
    dev_head_t : Transform | 3 items
    dig : Digitization | 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
    events : list | 1 items
    experimenter : str | 3 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    hpi_subsystem : dict | 2 items
    line_freq : float | 50.0
    lowpass : float | 356.3999938964844 Hz
    meas_date : tuple | 1970-01-01 00:00:00 GMT
    meas_id : dict | 4 items
    nchan : int | 404
    proc_history : list | 1 items
    proj_id : ndarray | 1 items
    proj_name : str | 11 items
    projs : list | 0 items
    sfreq : flo

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How many channels do you have for each type of sensors?</li>
    <li>What is the sampling frequency?</li>
    <li>Have the data been filtered?</li>
    <li>What is the frequency of the line noise?</li>
    <li>Is there any bad channel?</li>
    </ul>
</div>

INSERT ANSWERS HERE

raw.info is just a dictionary:

In [14]:
isinstance(raw.info, dict)

True

So we can access its elements this way:

In [15]:
raw.info['sfreq']  # Sampling frequency

1100.0

In [16]:
raw.info['bads']  # list of marked bad channels

[]

In [17]:
raw.info['line_freq']

50.0

The powerline frequency. 50 Hz in Sg; 60 Hz in Taiwan.

Next let's see what channels are present. It is available via the `raw.ch_names` attribute.

In [18]:
type(raw.ch_names)

list

In [19]:
raw.ch_names[:10]

['MEG0113',
 'MEG0112',
 'MEG0111',
 'MEG0122',
 'MEG0123',
 'MEG0121',
 'MEG0132',
 'MEG0133',
 'MEG0131',
 'MEG0143']

You can index it as a list

In [20]:
raw.ch_names[42]

'MEG0432'

In [21]:
raw.ch_names[:3]

['MEG0113', 'MEG0112', 'MEG0111']

Channel type of a specific channel

In [22]:
channel_type = mne.io.pick.channel_type(raw.info, 75)
print('Channel #75 is of type:', channel_type)

channel_type = mne.io.pick.channel_type(raw.info, 320)
print('Channel #320 is of type:', channel_type)

Channel #75 is of type: grad
Channel #320 is of type: eeg


Info contains all the details about the sensors (type, locations, coordinate frame etc.)

In [23]:
len(raw.info['chs'])

404

In [24]:
type(raw.info['chs'])

list

In [25]:
raw.info['chs'][0]

{'scanno': 1,
 'logno': 113,
 'kind': 1,
 'range': 1.9073486328125e-05,
 'cal': 3.250000046861601e-09,
 'coil_type': 3012,
 'loc': array([-0.1066    ,  0.0464    , -0.0604    , -0.01532829,  0.00619847,
        -0.99986327, -0.18597366, -0.98255992, -0.00331254, -0.98243302,
         0.185894  ,  0.016216  ]),
 'unit': 201,
 'unit_mul': 0,
 'ch_name': 'MEG0113',
 'coord_frame': 1 (FIFFV_COORD_DEVICE)}

In [26]:
raw.info['chs'][75]  # gradiometer

{'scanno': 76,
 'logno': 723,
 'kind': 1,
 'range': 1.9073486328125e-05,
 'cal': 3.250000046861601e-09,
 'coil_type': 3012,
 'loc': array([ 0.0186    ,  0.0105    ,  0.1096    ,  0.98852772, -0.00725508,
        -0.15086725,  0.00208091,  0.99939197, -0.03435012,  0.151026  ,
         0.033647  ,  0.98795599]),
 'unit': 201,
 'unit_mul': 0,
 'ch_name': 'MEG0723',
 'coord_frame': 1 (FIFFV_COORD_DEVICE)}

In [27]:
raw.info['chs'][330]

{'scanno': 331,
 'logno': 25,
 'kind': 2,
 'range': 0.00030517578125,
 'cal': 0.00019999999494757503,
 'coil_type': 1,
 'loc': array([ 5.63842431e-02,  3.68367434e-02,  9.40217227e-02,  8.26010015e-04,
         1.14762366e-01, -2.10680366e-02,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]),
 'unit': 107,
 'unit_mul': 0,
 'ch_name': 'EEG025',
 'coord_frame': 4 (FIFFV_COORD_HEAD)}

In [28]:
%matplotlib qt
raw.plot_sensors(kind='topomap', ch_type='grad');

In [29]:
#%matplotlib inline
raw.plot_sensors(kind='topomap', ch_type='mag');

In [30]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

### Setting channel types

Some channels are wrongly defined as EEG in the file. 2 of these are EOG (EEG061 and EEG062) and EEG063 is actually an ECG channel. EEG064 was recording but not connected to anything, so we'll [not] make it `'misc'`. We will now set the channel types. This will be useful for automatic artifact rejection.

In [31]:
raw.set_channel_types?

Signature: raw.set_channel_types(mapping)
Docstring:
Define the sensor type of channels.

Note: The following sensor types are accepted:
    ecg, eeg, emg, eog, exci, ias, misc, resp, seeg, stim, syst, ecog,
    hbo, hbr

Parameters
----------
mapping : dict
    a dictionary mapping a channel to a sensor type (str)
    {'EEG061': 'eog'}.

Notes
-----
.. versionadded:: 0.9.0
File:      c:\users\egora\anaconda3\envs\mne\lib\site-packages\mne\channels\channels.py
Type:      method


In [32]:
raw.set_channel_types({'EEG061': 'eog',
                       'EEG062': 'eog',
                       'EEG063': 'ecg',
                       'EEG064': 'misc'})  # EEG064 free-floating el.

raw.rename_channels({'EEG061': 'EOG061',
                     'EEG062': 'EOG062',
                     'EEG063': 'ECG063'})

In [33]:
indices = [i for i, s in enumerate(raw.info['ch_names']) if 'EOG061' in s]
indices

[366]

In [34]:
raw.info

-908196946


<Info | 27 non-empty fields
    acq_pars : str | 21833 items
    bads : list | 0 items
    ch_names : list | MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, ...
    chs : list | 404 items (GRAD: 204, MAG: 102, EEG: 70, EOG: 2, ECG: 1, MISC: 13, STIM: 3, CHPI: 9)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 36 items
    dev_head_t : Transform | 3 items
    dig : Digitization | 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
    events : list | 1 items
    experimenter : str | 3 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    hpi_subsystem : dict | 2 items
    line_freq : float | 50.0
    lowpass : float | 356.3999938964844 Hz
    meas_date : tuple | 1970-01-01 00:00:00 GMT
    meas_id : dict | 4 items
    nchan : int | 404
    proc_history : list | 1 items
    proj_id : ndarray | 1 items
    proj_name : str | 11 items
    projs : list | 0 items
    sfreq 

In [35]:
raw.plot_sensors(kind='topomap', ch_type='eeg');

## Accessing the data

To access the data just use the [] syntax as to access any element of a list, dict etc.

In [36]:
start, stop = 0, 10
data, times = raw[:, start:stop]  # fetch all channels and the first 10 time points
print(data.shape)
print(times.shape)

(404, 10)
(10,)


In [37]:
times  # always starts at 0 by convention

array([0.        , 0.00090909, 0.00181818, 0.00272727, 0.00363636,
       0.00454545, 0.00545455, 0.00636364, 0.00727273, 0.00818182])

Note that `raw[]` returns both the data and the times array.

# Resampling the data

We will now change the sampling frequency of the data to speed up the computations.

In [38]:
raw.resample?

Signature:
raw.resample(
    sfreq,
    npad='auto',
    window='boxcar',
    stim_picks=None,
    n_jobs=1,
    events=None,
    pad='reflect_limited',
    verbose=None,
)
Docstring:
Resample all channels.

The Raw object has to have the data loaded e.g. with ``preload=True``
or ``self.load_data()``.

.. warning:: The intended purpose of this function is primarily to
             speed up computations (e.g., projection calculation) when
             precise timing of events is not required, as downsampling
             raw data effectively jitters trigger timings. It is
             generally recommended not to epoch downsampled data,
             but instead epoch and then downsample, as epoching
             downsampled data jitters triggers.
             For more, see
             `this illustrative gist
             <https://gist.github.com/larsoner/01642cb3789992fbca59>`_.

             If resampling the continuous data is desired, it is
             recommended to construct even

And let's remove the unecessary channels

In [39]:
raw.load_data()  # it is required to load data in memory
raw.resample(300)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 404 x 147300 (491.0 sec), ~461.4 MB, data loaded>

In [40]:
raw.drop_channels?

Signature: raw.drop_channels(ch_names)
Docstring:
Drop channel(s).

Parameters
----------
ch_names : iterable or str
    Iterable (e.g. list) of channel name(s) or channel name to remove.

Returns
-------
inst : instance of Raw, Epochs, or Evoked
    The modified instance.

See Also
--------
reorder_channels
pick_channels
pick_types

Notes
-----
.. versionadded:: 0.9.0
File:      c:\users\egora\anaconda3\envs\mne\lib\site-packages\mne\channels\channels.py
Type:      method


In [41]:
to_drop = ['STI201', 'STI301', 'MISC201', 'MISC202', 'MISC203',
           'MISC204', 'MISC205', 'MISC206', 'MISC301', 'MISC302',
           'MISC303', 'MISC304', 'MISC305', 'MISC306', 'CHPI001',
           'CHPI002', 'CHPI003', 'CHPI004', 'CHPI005', 'CHPI006',
           'CHPI007', 'CHPI008', 'CHPI009']

In [42]:
raw.drop_channels(to_drop)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 381 x 147300 (491.0 sec), ~435.5 MB, data loaded>

In [43]:
raw.info

-908196946


<Info | 27 non-empty fields
    acq_pars : str | 21833 items
    bads : list | 0 items
    ch_names : list | MEG0113, MEG0112, MEG0111, MEG0122, MEG0123, MEG0121, ...
    chs : list | 381 items (GRAD: 204, MAG: 102, EEG: 70, EOG: 2, ECG: 1, MISC: 1, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 36 items
    dev_head_t : Transform | 3 items
    dig : Digitization | 137 items (3 Cardinal, 5 HPI, 75 EEG, 54 Extra)
    events : list | 1 items
    experimenter : str | 3 items
    file_id : dict | 4 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 1 items
    hpi_results : list | 1 items
    hpi_subsystem : dict | 2 items
    line_freq : float | 50.0
    lowpass : float | 150.0 Hz
    meas_date : tuple | 1970-01-01 00:00:00 GMT
    meas_id : dict | 4 items
    nchan : int | 381
    proc_history : list | 1 items
    proj_id : ndarray | 1 items
    proj_name : str | 11 items
    projs : list | 0 items
    sfreq : float | 300.0 Hz
   

# Visualizing raw data

See https://mne.tools/0.16/auto_tutorials/plot_visualize_raw.html
for more details.

Let's look at how to:
- browse data
- turn On/Off the PCA/SSP projections
- mark bad segments to obtained annotations
- group channel by types
- group channel by location

In [44]:
raw.plot?

Signature:
raw.plot(
    events=None,
    duration=10.0,
    start=0.0,
    n_channels=20,
    bgcolor='w',
    color=None,
    bad_color=(0.8, 0.8, 0.8),
    event_color='cyan',
    scalings=None,
    remove_dc=True,
    order=None,
    show_options=False,
    title=None,
    show=True,
    block=False,
    highpass=None,
    lowpass=None,
    filtorder=4,
    clipping=None,
    show_first_samp=False,
    proj=True,
    group_by='type',
    butterfly=False,
    decim='auto',
    noise_cov=None,
    event_id=None,
    show_scrollbars=True,
    verbose=None,
)
Docstring:
Plot raw data.

Parameters
----------
events : array | None
    Events to show with vertical bars.
duration : float
    Time window (s) to plot. The lesser of this value and the duration
    of the raw file will be used.
start : float
    Initial time to show (can be changed dynamically once plotted). If
    show_first_samp is True, then it is taken relative to
    ``raw.first_samp``.
n_channels : int
    Number of chan

In [45]:
#%matplotlib qt
raw.plot();

'a' for annotation to mark bad timepoints
'b' for butterfly plot

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Do you see any bad channel?</li>
    <li>Do you see any bad segment of data?</li>
    <li>Do you see any more then EOG blinks?</li>
    </ul>
</div>

In [46]:
#raw.annotations

In [47]:
#raw.annotations.save('annot.csv')

In [48]:
#!cat annot.csv

### Filtering

In [49]:
raw.filter?

Signature:
raw.filter(
    l_freq,
    h_freq,
    picks=None,
    filter_length='auto',
    l_trans_bandwidth='auto',
    h_trans_bandwidth='auto',
    n_jobs=1,
    method='fir',
    iir_params=None,
    phase='zero',
    fir_window='hamming',
    fir_design='firwin',
    skip_by_annotation=('edge', 'bad_acq_skip'),
    pad='reflect_limited',
    verbose=None,
)
Docstring:
Filter a subset of channels.

Parameters
----------

l_freq : float | None
    For FIR filters, the lower pass-band edge; for IIR filters, the upper
    cutoff frequency. If None the data are only low-passed.

h_freq : float | None
    For FIR filters, the upper pass-band edge; for IIR filters, the upper
    cutoff frequency. If None the data are only low-passed.
picks : str | list | slice | None
    Channels to include. Slices and lists of integers will be
    interpreted as channel indices. In lists, channel *type* strings
    (e.g., ``['meg', 'eeg']``) will pick channels of those
    types, channel *name* string

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Filter the raw data between 0Hz and 40Hz.</li>
    </ul>
</div>

In [50]:
raw.filter(0, 40)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 381 x 147300 (491.0 sec), ~435.5 MB, data loaded>

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Plot the 10 first seconds of stimulation channel just using matplotlib.</li>
    </ul>
</div>

Tips:

- Pick the stim channel using `mne.pick_types`
- Get the data for this channel
- Plot it using `plt.plot`

In [51]:
raw.info['sfreq']

300.0

In [52]:
data = raw.get_data('stim', start=0, stop=int(50 * raw.info['sfreq']))

In [53]:
# d = raw.get_data(picks=['EEG001', 'EEG002'])
# d = raw.get_data(picks=('eeg', 'mag'))
d = raw.get_data(picks=('grad',))
np.max(d)

2.385637307611378e-10

In [54]:
#### TODO
start = 0
stop = int(50 * raw.info['sfreq'])
data = raw.get_data('STI101', start=start, stop=stop)
data.shape

(1, 15000)

This yields 50, not 10, seconds.

In [55]:
raw.times[start:stop].shape

(15000,)

In [56]:
#%matplotlib inline

plt.plot(raw.times[start:stop], data.T)

In [57]:
np.max(data)

4101.0

## Define and read epochs

First extract events:

In [58]:
events = mne.find_events(raw, stim_channel='STI101', verbose=True)

259 events found
Event IDs: [   5    6    7   13   14   15   17   18   19  256  261  262  263  269
  270  271  273  274  275 4096 4101 4102 4103 4109 4110 4111 4113 4114
 4115 4352]


<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>What is the type of the variable events?</li>
    <li>What is the meaning of the 3 columnes of events?</li>
    <li>How many events of type 5 do you see?</li>
    </ul>
</div>

In [59]:
events[0:5, :]

array([[75052,     0,    13],
       [75336,     0,   256],
       [75964,     0,    14],
       [76158,    14,   270],
       [76896,     0,    13]], dtype=int64)

In [60]:
mne.find_events?

Signature:
mne.find_events(
    raw,
    stim_channel=None,
    output='onset',
    consecutive='increasing',
    min_duration=0,
    shortest_event=2,
    mask=None,
    uint_cast=False,
    mask_type='and',
    initial_event=False,
    verbose=None,
)
Docstring:
Find events from raw file.

See :ref:`tut_epoching_and_averaging` as well as :ref:`ex-read-events`
for more information about events.

Parameters
----------
raw : Raw object
    The raw data.
stim_channel : None | string | list of string
    Name of the stim channel or all the stim channels
    affected by triggers. If None, the config variables
    'MNE_STIM_CHANNEL', 'MNE_STIM_CHANNEL_1', 'MNE_STIM_CHANNEL_2',
    etc. are read. If these are not found, it will fall back to
    'STI 014' if present, then fall back to the first channel of type
    'stim', if present. If multiple channels are provided
    then the returned events are the union of all the events
    extracted from individual stim channels.
output : 'onset' | 'o

In [61]:
# events(:, 3) == 5 # matlab
np.sum(events[:, 2] == 5)

25

There was a time offset of 34.5ms in the stimulus presentation. We need to correct events accordingly.

In [62]:
delay = int(round(0.0345 * raw.info['sfreq']))
events[:, 0] = events[:, 0] + delay

Let's visualize the paradigm:

In [63]:
events = events[events[:, 2] < 20] # take only events with code less than 20

In [64]:
fig = mne.viz.plot_events(events, raw.info['sfreq']);

For event trigger and conditions we use a Python dictionary with keys that contain "/" for grouping sub-conditions

In [65]:
event_id = {
    'face/famous/first': 5,
    'face/famous/immediate': 6,
    'face/famous/long': 7,
    'face/unfamiliar/first': 13,
    'face/unfamiliar/immediate': 14,
    'face/unfamiliar/long': 15,
    'scrambled/first': 17,
    'scrambled/immediate': 18,
    'scrambled/long': 19,
}

In [66]:
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'],
                          event_id=event_id);

In [67]:
%matplotlib qt

raw.plot(event_id=event_id, events=events);

Define epochs parameters:

In [68]:
tmin = -0.5  # start of each epoch (500ms before the trigger)
tmax = 2.0  # end of each epoch (2000ms after the trigger)

Define the baseline period:

In [69]:
baseline = (-0.2, 0)  # means from 200ms before to stim onset (t = 0)

Define peak-to-peak (amplitude range) rejection parameters for gradiometers, magnetometers and EOG:

In [70]:
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)  # this can be highly data dependent

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>The <a href="https://autoreject.github.io/">autoreject</a> project aims to solve this problem of reject parameter setting. See the <a href="https://www.sciencedirect.com/science/article/pii/S1053811917305013">paper</a>.</li>
    </ul>
</div>

In [71]:
indices = [i for i, s in enumerate(raw.info['ch_names']) if 'STI101' in s]
indices

[380]

In [72]:
# we are picky again, this time with EOG
picks = mne.pick_types(raw.info, meg=True, eeg=True, eog=True,
                       stim=False, exclude='bads')

In [73]:
picks

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

Extract epochs:

In [74]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks, baseline=baseline,
                    reject=reject)

In [75]:
print(epochs)

<Epochs  |   146 events (good & bad), -0.5 - 2 sec, baseline [-0.2, 0], ~7.3 MB, data not loaded,
 'face/famous/first': 25
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 12
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 14
 'scrambled/long': 11>


In [76]:
epochs.drop_bad()  # remove bad epochs based on reject

<Epochs  |   79 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.3 MB, data not loaded,
 'face/famous/first': 13
 'face/famous/immediate': 3
 'face/famous/long': 6
 'face/unfamiliar/first': 17
 'face/unfamiliar/immediate': 4
 'face/unfamiliar/long': 6
 'scrambled/first': 15
 'scrambled/immediate': 9
 'scrambled/long': 6>

In [77]:
epochs.drop_bad?

Signature: epochs.drop_bad(reject='existing', flat='existing', verbose=None)
Docstring:
Drop bad epochs without retaining the epochs data.

Should be used before slicing operations.

.. warning:: This operation is slow since all epochs have to be read
             from disk. To avoid reading epochs from disk multiple
             times, use :meth:`mne.Epochs.load_data()`.

Parameters
----------
reject : dict | str | None
    Rejection parameters based on peak-to-peak amplitude.
    Valid keys are 'grad' | 'mag' | 'eeg' | 'eog' | 'ecg'.
    If reject is None then no rejection is done. If 'existing',
    then the rejection parameters set at instantiation are used.
flat : dict | str | None
    Rejection parameters based on flatness of signal.
    Valid keys are 'grad' | 'mag' | 'eeg' | 'eog' | 'ecg', and values
    are floats that set the minimum acceptable peak-to-peak amplitude.
    If flat is None then no rejection is done. If 'existing',
    then the flat parameters set at instantiati

In [78]:
epochs.load_data()  # load data in memory

<Epochs  |   79 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~178.4 MB, data loaded,
 'face/famous/first': 13
 'face/famous/immediate': 3
 'face/famous/long': 6
 'face/unfamiliar/first': 17
 'face/unfamiliar/immediate': 4
 'face/unfamiliar/long': 6
 'scrambled/first': 15
 'scrambled/immediate': 9
 'scrambled/long': 6>

Explore the epochs namespace

Hit ``epochs.<TAB>``

In [79]:
# epochs.

See how many epochs were dropped

CAUTION!
It's important to *keep* EOG channels despite their impact on rejection rate (almost half). (2020-01-03)

In [144]:
#%matplotlib inline

epochs.plot_drop_log();

In [81]:
for drop_log in epochs.drop_log[:20]:
    print(drop_log)

[]
[]
[]
[]
[]
[]
[]
['EOG062']
['EOG062']
[]
['EOG062']
[]
[]
['EOG062']
[]
[]
[]
[]
[]
[]


In [82]:
epochs.copy().drop(10, reason="I don't like this one").plot_drop_log();

In [83]:
epochs.copy().drop(10, reason="I don't like this one").drop_log[:20]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 ['EOG062'],
 ['EOG062'],
 [],
 ['EOG062'],
 [],
 [],
 ['EOG062'],
 ["I don't like this one"],
 [],
 [],
 [],
 [],
 []]

In [84]:
raw.filter(0, 80)

<Raw  |  sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif, n_channels x n_times : 381 x 147300 (491.0 sec), ~435.5 MB, data loaded>

In [85]:
events.shape

(146, 3)

In [86]:
epochs.events.shape

(79, 3)

In [87]:
events[epochs.selection] == epochs.events  # iow, epochs.events contains the kept epochs (post-rejection)

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ T

### Wait a second, did we just loose half of our epochs due to EOG???

We can probably do better. Let's use the PCA-based signal space projection (SSP) to regress out spatial patterns related to EOG and other offenders, ie., ECG.

Here is the workflow, we'll first detect EOG artifacts and visualize their impact. Then we'll compute related spatial patterns to mitigate these artifacts.

In [88]:
mne.preprocessing.create_eog_epochs?

Signature:
mne.preprocessing.create_eog_epochs(
    raw,
    ch_name=None,
    event_id=998,
    picks=None,
    tmin=-0.5,
    tmax=0.5,
    l_freq=1,
    h_freq=10,
    reject=None,
    flat=None,
    baseline=None,
    preload=True,
    reject_by_annotation=True,
    thresh=None,
    verbose=None,
)
Docstring:
Conveniently generate epochs around EOG artifact events.

Parameters
----------
raw : instance of Raw
    The raw data
ch_name : str
    The name of the channel to use for EOG peak detection.
    The argument is mandatory if the dataset contains no EOG channels.
event_id : int
    The index to assign to found events
picks : str | list | slice | None
    Channels to include. Slices and lists of integers will be
    interpreted as channel indices. In lists, channel *type* strings
    (e.g., ``['meg', 'eeg']``) will pick channels of those
    types, channel *name* strings (e.g., ``['MEG0111', 'MEG2623']``
    will pick the given channels. Can also be the string values
    "all" t

In [89]:
#%matplotlib inline

# We can use a convenience function
eog_epochs = mne.preprocessing.create_eog_epochs(raw.copy().filter(1, None))
eog_epochs.average().plot_joint()

[<Figure size 800x420 with 7 Axes>,
 <Figure size 800x420 with 7 Axes>,
 <Figure size 800x420 with 7 Axes>]

In [90]:
raw.plot(events=eog_epochs.events);

Compare the y axes to the ERF/ERPs we just saw. We face important degrees of contamination!

In [91]:
projs_eog, _ = mne.preprocessing.compute_proj_eog(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)

In [93]:
projs_eog

[<Projection  |  EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 204>,
 <Projection  |  EOG-planar--0.200-0.200-PCA-02, active : False, n_channels : 204>,
 <Projection  |  EOG-planar--0.200-0.200-PCA-03, active : False, n_channels : 204>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-02, active : False, n_channels : 102>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-03, active : False, n_channels : 102>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 70>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-02, active : False, n_channels : 70>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-03, active : False, n_channels : 70>]

In [94]:
#%matplotlib inline

layouts = [mne.find_layout(raw.info, ch_type=ch) for ch in ("eeg", "mag", "grad")]
mne.viz.plot_projs_topomap(projs_eog, layout=layouts);

Now the important question is how many components one should keep? Pro-tip: some of them don't look like clear artifact patterns. The good news is that we don't need to decide __*right*__ now.

In [95]:
# same business, same issue for ECG
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw.copy().filter(1, None))
ecg_epochs.average().plot_joint()

[<Figure size 800x420 with 7 Axes>,
 <Figure size 800x420 with 7 Axes>,
 <Figure size 800x420 with 7 Axes>]

We also face important insults from the cardiac signal... we'll project that out.

In [96]:
#%matplotlib inline

projs_ecg, _ = mne.preprocessing.compute_proj_ecg(
    raw, n_mag=3, n_grad=3, n_eeg=3, average=True)
mne.viz.plot_projs_topomap(projs_ecg, layout=layouts);

In [132]:
# now let's see how that would theoretically improve data preservation
reject2 = dict(mag=reject['mag'], grad=reject['grad']) 

epochs_clean = mne.Epochs(raw, events, event_id, tmin, tmax, proj=False,
                          picks=picks, baseline=baseline,
                          preload=False,
                          reject=reject2)

epochs_clean.add_proj(projs_eog + projs_ecg)
#epochs_clean.copy().apply_proj().average().plot(spatial_colors=True);  # apply projs on a copy

<Epochs  |   146 events (good & bad), -0.5 - 2 sec, baseline [-0.2, 0], ~7.5 MB, data not loaded,
 'face/famous/first': 25
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 12
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 14
 'scrambled/long': 11>

In [133]:
#%matplotlib inline

epochs_clean.copy().average().plot(spatial_colors=True);

In [134]:
#%matplotlib qt

epochs_clean.copy().average().plot(proj='interactive', spatial_colors=True);  # apply projs on a copy

now we keep all trials, probably we also removed some good signals.
we will postpone the selection of SSP vectors to later study the impact on
source localization

In [161]:
#%matplotlib inline

epochs_clean.average().plot(proj=True, spatial_colors=True);

<div class="alert alert-info">
    <b>REMARK</b>:
     <ul>
    <li>MNE keeps SSP projections inside the info and allows to apply them later.</li>
    </ul>
</div>

In [136]:
# let's overwrite
epochs = epochs_clean

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>Use ICA instead of SSP to remove artifacts</li>
    <li>What are potential benefits or disadvantages?</li>
    </ul>
</div>

### Visualization Epochs

See [this page](https://mne.tools/stable/auto_tutorials/epochs/plot_visualize_epochs.html) for options on how to visualize epochs.

Here is just an illustration to make a so-called ERP/ERF image:

In [137]:
raw.plot_psd(fmax=40);

In [138]:
epochs.plot_image(picks='EEG065', sigma=1.);

In [139]:
import matplotlib.pyplot as plt
plt.close('all')

In [142]:
epochs.plot();

In [143]:
print(epochs.drop_log)

[[], ['USER'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['MEG0121'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


### The epochs object is your MNE swiss army knife for processing segmented data!

- specialized methods for diagnostic plotting of data
- averaging
- saving
- manipulating data, e.g., rearranging or deleting single trials, resampling

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How could you get the epochs corresponding to face?</li>
    <li>How could you get the epochs corresponding to a familiar face?</li>
    <li>How could you get the epochs corresponding to a scrambled face?</li>
    </ul>
</div>

In [107]:
epochs.event_id

{'face/famous/first': 5,
 'face/famous/immediate': 6,
 'face/famous/long': 7,
 'face/unfamiliar/first': 13,
 'face/unfamiliar/immediate': 14,
 'face/unfamiliar/long': 15,
 'scrambled/first': 17,
 'scrambled/immediate': 18,
 'scrambled/long': 19}

In [108]:
epochs[['unfamiliar', 'scrambled']]

<Epochs  |   96 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.5 MB, data not loaded,
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 12
 'face/unfamiliar/long': 10
 'scrambled/first': 25
 'scrambled/immediate': 14
 'scrambled/long': 10>

## basic IO 

The standard scenario is saving the epochs into .fif file together with all the header data.

In [145]:
epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')
epochs_fname

'C:\\Users\\egora\\Downloads\\meeg\\ds000117-practical\\derivatives\\meg_derivatives\\sub-01\\ses-meg\\meg\\sub-01_ses-meg_task-facerecognition_run-01_proc-sss-epo.fif'

In [146]:
epochs.save(epochs_fname, overwrite=True)  # note that epochs are save in files ending with -epo.fif

Overwriting existing file.
Loading data for 1 events and 751 original time points ...
Loading data for 144 events and 751 original time points ...


In [148]:
data = epochs.get_data()
data.shape # chn, epo, samples

(144, 378, 751)

Scipy also supports reading and writing of matlab files. You can save your single trials with:

In [149]:
from scipy import io
epochs_data = epochs.get_data()
print(epochs_data.shape)
io.savemat('epochs_data.mat', dict(epochs_data=epochs_data),
           oned_as='row')

(144, 378, 751)


## Average the epochs to get ERF/ERP and plot it!

In [181]:
evoked = epochs.average()
print(evoked)

<Evoked  |  '0.17 * face/famous/first + 0.07 * face/famous/immediate + 0.10 * face/famous/long + 0.17 * face/unfamiliar/first + 0.08 * face/unfamiliar/immediate + 0.07 * face/unfamiliar/long + 0.17 * scrambled/first + 0.10 * scrambled/immediate + 0.08 * scrambled/long' (average, N=144), [-0.5, 2] sec, 376 ch, ~9.7 MB>


In [183]:
times = [0.0, 0.1, 0.18]  ## this is before the exclusion of the first PCA component from all three sensor types (compare to after; see below)
evoked.plot_topomap(ch_type='mag', times=times, proj=True);
evoked.plot_topomap(ch_type='grad', times=times, proj=True);
evoked.plot_topomap(ch_type='eeg', times=times, proj=True);

In [176]:
projs_eog[::3]

[<Projection  |  EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 204>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 70>]

In [177]:
list(projs_eog[i] for i in [0, 1, 3, 4, 6, 7])  # allows a custom selection of PCA components for exclusion

[<Projection  |  EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 204>,
 <Projection  |  EOG-planar--0.200-0.200-PCA-02, active : False, n_channels : 204>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>,
 <Projection  |  EOG-axial--0.200-0.200-PCA-02, active : False, n_channels : 102>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 70>,
 <Projection  |  EOG-eeg--0.200-0.200-PCA-02, active : False, n_channels : 70>]

In [192]:
# refresh evoked
evoked = epochs.average()
evoked.del_proj()  # delete previous proj
# take first for each sensor type
evoked.add_proj(projs_eog[::3] + projs_ecg[::3])  # selecting every third PCA component, starting with the first one
evoked.apply_proj()  # apply

<Evoked  |  '0.17 * face/famous/first + 0.07 * face/famous/immediate + 0.10 * face/famous/long + 0.17 * face/unfamiliar/first + 0.08 * face/unfamiliar/immediate + 0.07 * face/unfamiliar/long + 0.17 * scrambled/first + 0.10 * scrambled/immediate + 0.08 * scrambled/long' (average, N=144), [-0.5, 2] sec, 376 ch, ~9.6 MB>

In [193]:
#plt.close('all')
evoked.plot(proj=True);

We can also show sensor position as line color:

In [194]:
evoked.plot(spatial_colors=True, proj=True);  # note the legend

In [195]:
times = [0.0, 0.1, 0.18]
evoked.plot_topomap(ch_type='mag', times=times, proj=True);
evoked.plot_topomap(ch_type='grad', times=times, proj=True);
evoked.plot_topomap(ch_type='eeg', times=times, proj=True);

In [196]:
import numpy as np
# pure topography plots called topomap in the MNE jargon

The following plots only react to commands related to the channel type imaged by the topomap -- tick-marking the other channel PCA components does nothing.

In [188]:
#for ch_type in ('mag', 'grad', 'eeg'):
    #evoked.plot_topomap(times=np.linspace(0.05, 0.45, 8),
                        #ch_type=ch_type, proj=True);

In [187]:
evoked.plot_topomap(times=np.linspace(0.05, 0.45, 8),
                        ch_type='mag', proj='interactive');  ## changed from 'True' to 'interactive'

In [ ]:
evoked.plot_topomap(times=np.linspace(0.05, 0.45, 8),
                        ch_type='grad', proj='interactive');

In [ ]:
evoked.plot_topomap(times=np.linspace(0.05, 0.45, 8),
                        ch_type='eeg', proj='interactive');

<div class="alert alert-success">
    <b>Exercise</b>:
     <ul>
    <li>How does SSP impact the evoked responses? Use proj="interactive" to explore</li>
    </ul>
</div>

Topoplot and time series can also be shown in one single plot:

In [200]:
evoked.plot_joint(times=[0.17]);

## Accessing and indexing epochs by condition

Epochs can be indexed by integers or slices to select a subset of epochs but also with strings to select by conditions `epochs[condition]`

In [201]:
epochs[0]  # first epoch

<Epochs  |   1 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.5 MB, data not loaded,
 'face/unfamiliar/first': 1>

In [202]:
epochs[:10]  # first 10 epochs

<Epochs  |   10 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.5 MB, data not loaded,
 'face/famous/first': 3
 'face/unfamiliar/first': 4
 'face/unfamiliar/long': 2
 'scrambled/first': 1>

In [203]:
epochs['face']  # epochs for a face

<Epochs  |   94 events (all good), -0.5 - 2 sec, baseline [-0.2, 0], ~7.5 MB, data not loaded,
 'face/famous/first': 24
 'face/famous/immediate': 10
 'face/famous/long': 14
 'face/unfamiliar/first': 25
 'face/unfamiliar/immediate': 11
 'face/unfamiliar/long': 10>

In event_id, "/" selects conditions in a hierarchical way, e.g. here, "face" vs. "scrambled", "famous" vs. "unfamiliar", and MNE can select them individually

In [204]:
epochs['face'].average().\
    pick_types(meg='grad').crop(-0.1, 0.25).plot(spatial_colors=True);

Apply this to visualize all the conditions in `event_id`

In [205]:
plt.close('all')
for condition in ['face', 'scrambled']:
    epochs[condition].average().plot_topomap(times=[0.1, 0.15], title=condition);

## Write evoked data to disk

In [206]:
evoked_fname = raw_fname.replace('_meg.fif', '-ave.fif')
evoked_fname

'C:\\Users\\egora\\Downloads\\meeg\\ds000117-practical\\derivatives\\meg_derivatives\\sub-01\\ses-meg\\meg\\sub-01_ses-meg_task-facerecognition_run-01_proc-sss-ave.fif'

In [215]:
evoked.save(evoked_fname)  # note that the file for evoked ends with -ave.fif

or to write multiple conditions in 1 file

In [216]:
evokeds_list = mne.read_evokeds(evoked_fname, baseline=(None, 0), proj=True)
evokeds_list

[<Evoked  |  '0.17 * face/famous/first + 0.07 * face/famous/immediate + 0.10 * face/famous/long + 0.17 * face/unfamiliar/first + 0.08 * face/unfamiliar/immediate + 0.07 * face/unfamiliar/long + 0.17 * scrambled/first + 0.10 * scrambled/immediate + 0.08 * scrambled/long' (average, N=144), [-0.5, 2] sec, 376 ch, ~9.5 MB>]

A single list is written/read.

A separate list is created per condition (compare to the above):

In [217]:
evokeds_list = [epochs[k].average() for k in event_id]  # get evokeds
mne.write_evokeds(evoked_fname, evokeds_list)

In [218]:
evokeds_list

[<Evoked  |  'face/famous/first' (average, N=24), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/famous/immediate' (average, N=10), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/famous/long' (average, N=14), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/unfamiliar/first' (average, N=25), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/unfamiliar/immediate' (average, N=11), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/unfamiliar/long' (average, N=10), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'scrambled/first' (average, N=25), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'scrambled/immediate' (average, N=14), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'scrambled/long' (average, N=11), [-0.5, 2] sec, 376 ch, ~9.7 MB>]

In [219]:
evokeds_list = mne.read_evokeds(evoked_fname, baseline=(None, 0), proj=True)
evokeds_list

[<Evoked  |  'face/famous/first' (average, N=24), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/famous/immediate' (average, N=10), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/famous/long' (average, N=14), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/unfamiliar/first' (average, N=25), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/unfamiliar/immediate' (average, N=11), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'face/unfamiliar/long' (average, N=10), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'scrambled/first' (average, N=25), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'scrambled/immediate' (average, N=14), [-0.5, 2] sec, 376 ch, ~9.7 MB>,
 <Evoked  |  'scrambled/long' (average, N=11), [-0.5, 2] sec, 376 ch, ~9.7 MB>]

### Reading evoked from disk

It is also possible to read evoked data stored in a fif file (see above).

Or give the explicit name of the averaged condition:

In [220]:
evoked1 = mne.read_evokeds(evoked_fname, condition="face/famous/first",
                           baseline=(None, 0), proj=True)

In [221]:
evoked1

<Evoked  |  'face/famous/first' (average, N=24), [-0.5, 2] sec, 376 ch, ~9.7 MB>

**Remark:** Did you notice that you can apply some preprocessing on reading the evokeds from disk?

### Compute a contrast:

In [222]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()

In [223]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])

Note that this combines evokeds taking into account the number of averaged epochs (to scale the noise variance)

In [224]:
print(evoked1.nave)  # average of 12 epochs
print(contrast.nave)  # average of 116 epochs

24
130.55555555555557


In [225]:
print(contrast)

<Evoked  |  '0.500 * 0.26 * face/famous/first + 0.11 * face/famous/immediate + 0.15 * face/famous/long + 0.27 * face/unfamiliar/first + 0.12 * face/unfamiliar/immediate + 0.11 * face/unfamiliar/long + -0.500 * 0.50 * scrambled/first + 0.28 * scrambled/immediate + 0.22 * scrambled/long' (average, N=130.55555555555557), [-0.5, 2] sec, 376 ch, ~9.7 MB>


In [226]:
fig = contrast.copy().pick('grad').crop(-0.1, 0.3).plot_joint()

In [227]:
fig = contrast.copy().pick('mag').crop(-0.1, 0.3).plot_joint()

In [228]:
fig = contrast.copy().pick('eeg').crop(-0.1, 0.3).plot_joint()

In [229]:
evoked_face

<Evoked  |  '0.26 * face/famous/first + 0.11 * face/famous/immediate + 0.15 * face/famous/long + 0.27 * face/unfamiliar/first + 0.12 * face/unfamiliar/immediate + 0.11 * face/unfamiliar/long' (average, N=94), [-0.5, 2] sec, 376 ch, ~9.7 MB>

In [230]:
evoked_scrambled

<Evoked  |  '0.50 * scrambled/first + 0.28 * scrambled/immediate + 0.22 * scrambled/long' (average, N=50), [-0.5, 2] sec, 376 ch, ~9.7 MB>

### Save your figure as pdf

In [232]:
%matplotlib qt
import numpy as np
contrast.plot_topomap(times=np.linspace(0.05, 0.15, 5), ch_type='mag')
plt.savefig('toto.pdf')
#!open toto.pdf  # works only on a mac

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Compute the evoked data for 'famous', 'unfamiliar', 'scrambled' faces</li>
      <li>Crop the data between -0.2s and 0.4s</li>     
      <li>Plot the channel EEG065 in all 3 conditions using mne.viz.plot_compare_evokeds function</li>
    </ul>
</div>

See: https://mne.tools/stable/generated/mne.viz.plot_compare_evokeds.html

In [233]:
evoked_famous = epochs['famous'].average().crop(-0.1, 0.4)
evoked_scrambled = epochs['scrambled'].average().crop(-0.1, 0.4)
evoked_unfamiliar = epochs['unfamiliar'].average().crop(-0.1, 0.4)

In [235]:
plt.close('all')
mne.viz.plot_evoked_topo([evoked_famous, evoked_scrambled, evoked_unfamiliar]);

In [237]:
evokeds = {k:epochs[k].average().crop(-0.1, 0.4)
           for k in ['famous', 'unfamiliar', 'scrambled']}

In [238]:
plt.close('all')
mne.viz.plot_compare_evokeds(evokeds, picks='EEG065');

## ADVANCED: Customize your plots

Want to have every text in blue?

In [ ]:
import matplotlib as mpl
fig = evoked1.plot(show=False)  # butterfly plots
fig.subplots_adjust(hspace=1.0)
for text in fig.findobj(mpl.text.Text):
    text.set_fontsize(18)
    text.set_color('blue')
for ax in fig.get_axes():
    ax.axvline(0., color='red', linestyle='--')
plt.tight_layout()
fig.savefig('plot_erf.pdf');